In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='coursebase_faqs.csv', encoding='cp1252', source_column='prompt')
data = loader.load()

In [7]:
'''
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: "
)

e = embeddings.embed_query("What is your refund policy")
'''

from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
e = hf.embed_query("What is your refund policy")

In [38]:
len(e)

768

In [21]:
from langchain_community.vectorstores.faiss import FAISS

vectordb = FAISS.from_documents(documents=data, embedding=hf)

In [23]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents("how about job support")
rdocs

[Document(id='1cb9658e-b567-4c17-82e4-3e373b0e2db5', metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
 Document(id='d9b0b405-5143-4c89-a54b-62b3d47f6e23', metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepar

In [106]:
# from langchain_deepseek import ChatDeepSeek
# import os

# DeepSeek gave error saying : Error code: 402 - {'error': {'message': 'Insufficient Balance'}} .
'''
os.environ['DEEPSEEK_API_KEY'] = 'sk-0aef4b3225de493caaf3db8c996a713b'

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)
'''

In [112]:
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
)



AIMessage(content="I don't know the specific details of your course or industry, but many courses offer job placement assistance or career services to help graduates find employment. It's also common for employers to look for candidates with relevant skills and experience gained through coursework. You can check with your course provider or career services department for more information on job prospects after completing your course.", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-27T08:22:28.7489Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2836208708, 'load_duration': 30250375, 'prompt_eval_count': 83, 'prompt_eval_duration': 877000000, 'eval_count': 71, 'eval_duration': 1926000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c335b52a-ba8b-4a92-a7f5-39ebbb72e53d-0', usage_metadata={'input_tokens': 83, 'output_tokens': 71, 'total_tokens': 154})

In [114]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

retriever = vectordb.as_retriever()
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "can i get a job after this course ?"})
print(response["answer"])

Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job. However, we want to be honest and do not want to make any impractical promises! We can't guarantee a specific job, but we're confident that our course will equip you with the necessary skills to increase your chances of getting hired.


In [116]:
response2 = rag_chain.invoke({"input": "Do you guys offer internship and also do you offer EMI payments?"})
response2

{'input': 'Do you guys offer internship and also do you offer EMI payments?',
 'context': [Document(id='3744f404-13cb-4df3-a114-34b7e9973826', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}, page_content='prompt: Do you provide any virtual internship?\nresponse: Yes'),
  Document(id='e2190ca7-e952-4614-b4b0-2596a69f6520', metadata={'source': 'Do we have an EMI option?', 'row': 13}, page_content='prompt: Do we have an EMI option?\nresponse: No'),
  Document(id='1cb9658e-b567-4c17-82e4-3e373b0e2db5', metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
  Document(id='d9b0b405-5143-4c89-a54b-62b3d47f6e23', metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='pro

In [118]:
response3 = rag_chain.invoke({"input":"do you have javascript course"})
response3

{'input': 'do you have javascript course',
 'context': [Document(id='4b1b45c5-8cfb-42e8-8e8b-57c398bfbb9d', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}, page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.'),
  Document(id='79520c3e-40ed-4f60-9703-5eec53cb4ccf', metadata={'source': 'I’m not sure if this course is good enough for me to invest some money. What can I do?', 'row': 20}, page_content='prompt: I’m not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Don’t worry. Many videos in this course are free so watch them to get an idea of the quality of teaching. Dhaval Patel (the c

In [120]:
response4 = rag_chain.invoke({"input":"do you have course on home cooking"})
response4

{'input': 'do you have course on home cooking',
 'context': [Document(id='117b1be8-3185-4e07-933f-8e9399c8d047', metadata={'source': '\nI don’t have a laptop, can I take this course?', 'row': 25}, page_content='prompt: I don’t have a laptop, can I take this course?\nresponse: We recommend learning by doing and therefore you need to have a laptop or a PC (at least 4 GB ram).'),
  Document(id='d9b0b405-5143-4c89-a54b-62b3d47f6e23', metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepa

In [ ]:
# The LLM we are using is not hallucinating. If it does not have info about a course it says "I don't know".
# This is happening because we are using Prompt templates which specifies if you don't know the answer, say don't know instead of hallucinating.
